# Completeness (Departmental)

These graphs show per collecting deparment, how many object records are field-complete (that is, if a concept grouping such as 'Depiction' has 10 possible fields such as contentPlace, contentPerson, etc, an object with 5 of them with values set will be 50% complete). 


This graphs per department, per conceptual grouping of fields, how many objects are field-complete (i.e. if a concept has 10 fields, an object with 5 of them with values will be 50% complete)

Two graphs, summaries and more details.

Notes:
    
  - Some sets of fields within the conceptual grouping only need to have one of the fields within the set to contain content, so they only count as 1. E.g. if there are 13 fields, of which 3 are in a set, the number of
    total number fields in the conceptual grouping will be 10. This punishes objects that do have information in all 3 fields in the set, but we cannot identify which should or should not have 1 or 3 set.
  - Some fields can count for more than one conceptual grouping (e.h. materialTechniques countrs for both MATERIALT and PRODUCTION).

In [1]:
import glob
import pandas as pd

objects_df = pd.read_hdf('/home/richard/Git/bbk-mqaf/data/latest/field.h5')

In [2]:
# First create new data frame reducing down to just the columns we need

complete_df = objects_df.filter(regex='collectionCode|completeness*', axis=1)

In [3]:
depts = complete_df[' collectionCode'].unique()

Then group by department and count objects for each concept showing number per 10% completeness
E.g. for PDP for PRODUCTION 0-10% 57000 object, 10-20% 87000, 20-30% 200,000 objects
Loop over collectioncode, find all unique types and filter out rest
Loop over each completness column, sum number that fall into each range
10 bar graphs per department, one for each concept showing amount

In [8]:
# Save all the possible collectioncode
import altair as alt

ranges = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
count =0

dept_col_counts = None
cur_depts = complete_df[' collectionCode'].unique()
for dept_code in depts:
  vcharts = []
  hcharts = []
  charts = []
  print(dept_code)
  dept_stats_df = complete_df.loc[complete_df[' collectionCode'] == dept_code]
  # Now need to loop over each completeness coll in turn
  for column in dept_stats_df.columns[1:]:  
    dept_col_counts = dept_stats_df[column].groupby(pd.cut(dept_stats_df[column], ranges, labels=["0-10%","10-20%","20-30%","30-40%","40-50%","50-60%","60-70%","70-80%","80-90%","90-100%"], include_lowest=True)).count()
    dept_col_df = pd.DataFrame(dept_col_counts)
    dept_col_df.columns = ['completeness']
    dept_col_df.rename_axis("percent")
    chart = alt.Chart(dept_col_df.reset_index()).mark_bar().encode(
      x=alt.X(column),
      y=alt.Y('completeness', scale=alt.Scale(domain=(0, 1000000)), axis=alt.Axis(title='Objects')))
    chart_text = chart.mark_text().encode(
        text='completeness:Q')
    charts.append((chart + chart_text))
    if count > 3:
      hcharts.append(alt.hconcat(*charts))
      charts = []
      count = 0
    else:
      count += 1
  break
alt.vconcat(*hcharts)

T&P


alt.VConcatChart(...)

In [6]:
import re
tidied_df = complete_df.rename(columns=lambda x: re.sub('completeness:','',x))

In [4]:
deptNames = {"APS": "Apsley House",
             "CER": "Ceramics Collection",
             "CIRC": "Circulation Department (1909 – 1977)",
             "DAD": "Design, Architecture & Digital Department",
             "EAS": "East Asia Collection",
             "EXH": "Exhibitions Department",
             "FoB": "Fabric of the Building",
             "FWK": "Furniture and Woodwork Collection",
             "MES": "Middle East Section",
             "MET": "Metalwork Collection",
             "MoC": "Museum of Childhood",
             "NAL": "National Art Library",
             "PDP": "Prints, Drawings & Paintings Collection",
             "RPS": "Royal Photographic Society Collection",
             "SCP": "Sculpture Collection",
             "SSEA": "South & South East Asia Collection",
             "T&F": "Textiles & Fashion Collection",
             "T&P": "Theatre & Performance Collection",
             "VAA": "V&A Archive Collection",
             "WED": "Wedgwood Collection",
             "AAD": "Archive of Art & Design",
             "DOP": "Department of Photography"}

In [9]:
import altair as alt

ranges = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
count =0

dept_col_counts = None
depts = tidied_df[' collectionCode'].unique()
charts = []
hcharts = []
for dept_code in depts:
  dept_rows_df = pd.DataFrame()
  dept_stats_df = tidied_df.loc[tidied_df[' collectionCode'] == dept_code]
    
  if(len(dept_stats_df) < 10):
    # Ignore EXH and other smaller departments
    continue
    
  # Now need to loop over each completeness coll in turn
  for column in dept_stats_df.columns[1:]:
    
    dept_col_counts = dept_stats_df[column].groupby(pd.cut(dept_stats_df[column], ranges, labels=["0-10%","10-20%","20-30%","30-40%","40-50%","50-60%","60-70%","70-80%","80-90%","90-100%"], include_lowest=True)).count()
    dept_col_counts.name = column
    dept_rows_df = dept_rows_df.append(dept_col_counts, ignore_index=False)
    
  dept_rows_df = dept_rows_df.rename_axis('Concept').rename_axis('Percentages', axis='columns')
    # TO handle converting from CategoricalIndex - may change https://github.com/pandas-dev/pandas/issues/19136
  dept_rows_df.columns = dept_rows_df.columns.tolist()
  dept_rows_df = dept_rows_df.reset_index()
  dept_rows_melt_df = dept_rows_df.melt(id_vars=['Concept'], var_name='Percentage', value_name='Objects')
  
  chart = alt.Chart(dept_rows_melt_df).mark_bar().encode(
       x=alt.X('Objects:Q', axis=alt.Axis(title='Objects')),
       y='Concept:O',
        color='Percentage:O',
       tooltip=['Objects', 'Percentage']
    ).properties(width=220, title="%s (%d)" % (deptNames[dept_code], len(dept_stats_df)))
  charts.append(chart)
  if count > 0:
      hcharts.append(alt.hconcat(*charts))
      charts = []
      count = 0
  else:
      count += 1
        
hcharts.append(alt.hconcat(*charts))

alt.vconcat(*hcharts)

alt.VConcatChart(...)